In [1]:
import tensorflow as tf
print(tf.__version__)
import os
import numpy as np

1.13.2


### To load the image and preprocess

In [2]:
def read_images(dataset_path):
    global classes
    im_path,labels=list(),list()
    classes=os.listdir(dataset_path)
    for i in classes:
        path=os.path.join(dataset_path,i)
        for j in os.listdir(path):
            path1=os.path.join(path,j)
            im_path.append(path1)
            lab=classes.index(i)
            if lab==0:
                labels.append([0,1])
            else:
                labels.append([1,0])
    for i in range(len(classes)):
        print('{}:{}'.format(classes[i],i))
    im_path=tf.convert_to_tensor(im_path)
    labels=tf.convert_to_tensor(labels)
    labels=tf.cast(labels,tf.float32)
    image,labels=tf.train.slice_input_producer([im_path,labels],shuffle=True)
    image = tf.read_file(image)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_images(image, [32, 32])
    # Normalize
    image = image * 1.0/127.5 - 1.0
    X, Y = tf.train.batch([image, labels], batch_size=32,capacity=32 * 8,num_threads=4)
    return X,Y

In [3]:
dataset_path='dataset/train/'   #give the dataset path
X,Y=read_images(dataset_path)

stone:0
paper:1
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions f

### CNN Architecture

1.conv layer 3x3 32 filters

2.conv layer 3 x3 64 filters

3.dense layer 120 neurons

4.dense layer 2 neurons


In [4]:
def getModel(x,re=False):
    print(x.shape)
    with tf.variable_scope('conv1',reuse=re) as scope:
        w=tf.get_variable('w',shape=[3,3,3,32],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('b',initializer=tf.zeros([32]))
        l1=tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='VALID')
        l1_b=tf.nn.bias_add(l1,b)
        l1_act=tf.nn.relu(l1_b)
    print(l1_act)
    with tf.variable_scope('conv2',reuse=re) as scope:
        w=tf.get_variable('w',shape=[3,3,32,64],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('b',initializer=tf.zeros([64]))
        l2=tf.nn.conv2d(l1_act,w,strides=[1,1,1,1],padding='VALID')
        l2_b=tf.nn.bias_add(l2,b)
        l2_act=tf.nn.relu(l2_b)
    l3_max=tf.nn.max_pool(l2_act,[1,2,2,1],strides=[1,2,2,1],padding='VALID')
    print(l3_max.shape)
    with tf.variable_scope('dense1',reuse=re) as scope:
        w=tf.get_variable('w',shape=[14*14*64,120],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('b',initializer=tf.zeros([120]))
        l4=tf.reshape(l3_max,[32,14*14*64])
        print('l4',l4.shape)
        l4=tf.matmul(l4,w)
        l4_b=tf.nn.bias_add(l4,b)
        l4_act=tf.nn.relu(l4_b)
    with tf.variable_scope('dense2',reuse=re) as scope:
        w=tf.get_variable('w',shape=[120,2],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('b',initializer=tf.zeros(2))
        l5=tf.matmul(l4_act,w)+b
    print(l5)
    
    return l5

### Load model Architecture

In [5]:
model_tr=getModel(X,re=False)   # if re is Fasle it initialize new value to the variables
model_te=getModel(X,re=True)    # if re is True it takes the already initialized values

(32, 32, 32, 3)

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Tensor("conv1/Relu:0", shape=(32, 30, 30, 32), dtype=float32)
(32, 14, 14, 64)
('l4', TensorShape([Dimension(32), Dimension(12544)]))
Tensor("dense2/add:0", shape=(32, 2), dtype=float32)
(32, 32, 32, 3)
Tensor("conv1_1/Relu:0", shape=(32, 30, 30, 32), dtype=float32)
(32, 14, 14, 64)
('l4', TensorShape([Dimension(32), Dimension(12544)]))
Tensor("dense2_1/add:0", shape=(32, 2), dtype=float32)


### Optimizer and Loss

In [6]:
eval_pred=tf.nn.softmax(model_tr)
val=tf.nn.softmax_cross_entropy_with_logits(logits=model_tr,labels=Y)
cost=tf.reduce_mean(val)
train_op=tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### Testing

In [7]:
a=tf.argmax(model_te,1)
b=tf.argmax(Y,1)
correct_pred=tf.equal(a,b)

In [8]:
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

### Save Model
Create a folder with the name of "model" in your working directory 

In [9]:
saver = tf.train.Saver()

In [10]:
ite=6466//32  # total no. of image divided by batch size
epoch=5  # epoch size
with tf.Session() as sess:
    coord=tf.train.Coordinator()
    threads=tf.train.start_queue_runners(coord=coord)
    sess.run(tf.global_variables_initializer())
    for epo in range(epoch):
        for it in range(ite):
            sess.run(train_op)
            if it%50 ==0:
                loss=sess.run(cost)
                acc=sess.run(accuracy)
                print('{}/{} Iteration:{} Loss: {:.3f} Accuracy : {}'.format(epo+1,epoch,it,loss,acc))
    saver.save(sess,'model/my_test_model')  # save model
    coord.request_stop()
    coord.join(threads)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
1/5 Iteration:0 Loss: 1.096 Accuracy : 0.3125
1/5 Iteration:50 Loss: 0.010 Accuracy : 1.0
1/5 Iteration:100 Loss: 0.055 Accuracy : 0.9375
1/5 Iteration:150 Loss: 0.001 Accuracy : 1.0
1/5 Iteration:200 Loss: 0.032 Accuracy : 1.0
2/5 Iteration:0 Loss: 0.000 Accuracy : 0.96875
2/5 Iteration:50 Loss: 0.003 Accuracy : 1.0
2/5 Iteration:100 Loss: 0.001 Accuracy : 0.96875
2/5 Iteration:150 Loss: 0.003 Accuracy : 1.0
2/5 Iteration:200 Loss: 0.000 Accuracy : 1.0
3/5 Iteration:0 Loss: 0.003 Accuracy : 1.0
3/5 Iteration:50 Loss: 0.000 Accuracy : 1.0
3/5 Iteration:100 Loss: 0.000 Accuracy : 1.0
3/5 Iteration:150 Loss: 0.000 Accuracy : 1.0
3/5 Iteration:200 Loss: 0.000 Accuracy : 1.0
4/5 Iteration:0 Loss: 0.000 Accuracy : 1.0
4/5 Iteration:50 Loss: 0.000 Accuracy : 1.0
4/5 Iteration:100 Loss: 0.000 Accuracy : 1.0
4/5 Iteration:150 Loss: 0.000 Accuracy : 1.0
4/5 Iteration:200 Loss: 0.000 Accuracy : 1.0
5/5 Iteration:0

### Restore the saved model 

In [11]:
with tf.Session() as sess:    
    saver = tf.train.import_meta_graph('model/my_test_model.meta')
    saver.restore(sess,tf.train.latest_checkpoint('model/'))
    for i in tf.trainable_variables():
        print(i)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from model/my_test_model
<tf.Variable 'conv1/w:0' shape=(3, 3, 3, 32) dtype=float32_ref>
<tf.Variable 'conv1/b:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'conv2/w:0' shape=(3, 3, 32, 64) dtype=float32_ref>
<tf.Variable 'conv2/b:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'dense1/w:0' shape=(12544, 120) dtype=float32_ref>
<tf.Variable 'dense1/b:0' shape=(120,) dtype=float32_ref>
<tf.Variable 'dense2/w:0' shape=(120, 2) dtype=float32_ref>
<tf.Variable 'dense2/b:0' shape=(2,) dtype=float32_ref>
<tf.Variable 'conv1/w:0' shape=(3, 3, 3, 32) dtype=float32_ref>
<tf.Variable 'conv1/b:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'conv2/w:0' shape=(3, 3, 32, 64) dtype=float32_ref>
<tf.Variable 'conv2/b:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'dense1/w:0' shape=(12544, 120) dtype=float32_ref>
<tf.Variable 'dense1/b:0' shape=(120,) dtype=float32_ref>
<tf.Varia